<h1>Análise Patrimonial das eleições </h1>



<h2>Proposta: Analisar a evolução patrimonial dos candidatos recorrentes na eleição de 2022 que se candidataram para as eleições de 2018 ou 2020</h2>


<h3>Pergunta Principal: Qual a evolução patrimonial dos candidatos recorrentes da eleição de 2022?</h3>

Aluno: **Juan Lima** [Linkedin](https://www.linkedin.com/in/juanrdlima/) [Email](mailto:juan.rdlima@gmail.com)

# Parte 3: 

In [0]:
#Importação das Bibliotecas necessárias

#Muitas Bibliotecas já vem incorporadas no Databricks, vou listar as utilizadas que não precisaram ser importadas como comentário

#import dbutils

# Data Engineering

%md
<h3>Gold Layer:</h3><br> Tabelas tratadas para serem usadas

O intuito desta etapa é deixar as tabelas o mais simples possível para a extração final de informações.

In [0]:
%sql
--Criação do schema de Silver Layer
CREATE SCHEMA IF NOT EXISTS  eleicao_gold
    COMMENT 'Gold Schema with ready to use tables ' 

### TABELAS FINAIS

Para as tabelas que em teoria seriam disponibilizadas para a análise iremos disponibilizar as tabelas particionadas por anos, com a informação de completa, as descritivas de cada candidato e a informação de seus bens abertos ou não por tipo.

São as tabelas:
1. Tabela de Candidatos (unicidade do "SQ_CANDIDATO") por ano com a descrição de cada candidato e seu patrimônio consolidado
2. Tabela de Patrimônio, abertas por tipo de bem declarado e ano de eleição.
3. Tabela relacional de código de tipo de bem e descritivo 
4. Tabela voltada para a análise com os candidatos da eleição de 2022 que também participaram da eleição de 2018 ou de 2020, com os respectivos patrimonios e informações descritivas.

**Tabela "De Para" relacional de tipo de bem declarado**:

In [0]:
#df "de para" relacional tipo bem
df = sqlContext.sql("""
            SELECT
                 DISTINCT
                 CAST(CD_TIPO_BEM_CANDIDATO AS BIGINT) AS CD_TIPO_BEM_CANDIDATO
                ,DS_TIPO_BEM_CANDIDATO
            FROM 
                eleicao_silver.bens_candidatos_tipo
            ORDER BY 
                1
    """ )
display(df)

CD_TIPO_BEM_CANDIDATO,DS_TIPO_BEM_CANDIDATO
1,Prédio residencial
2,Prédio comercial
3,Galpão
11,Apartamento
12,Casa
13,Terreno
14,Terra nua
15,Sala ou conjunto
16,Construção
17,Benfeitorias


In [0]:
%sql
CREATE TABLE eleicao_gold.tipo_bem ( 
     CD_TIPO_BEM_CANDIDATO BIGINT PRIMARY KEY COMMENT 'Unique identifier for each heritage asset kind of the candidates'
    ,DS_TIPO_BEM_CANDIDATO VARCHAR(50) COMMENT 'Description of the type of each heritage asset' 
);

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2412889709161111>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-2412889709161111>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("Q1JFQVRFIFRBQkxFIGVsZWljYW9fZ29sZC50aXBvX2JlbSAoIAogICAgIENEX1RJUE9fQkVNX0NBTkRJREFUTyBCSUdJTlQgUFJJTUFSWSBLRVkgQ09NTUVOVCAnVW5pcXVlIGlkZW50aWZpZXIgZm9yIGVhY2ggaGVyaXRhZ2UgYXNzZXQga2luZCBvZiB0aGUgY2FuZGlkYXRlcycKICAgICxEU19USVBPX0JFTV9DQU5ESURBVE8gVkFSQ0hBUig1MCkgQ09NTUVOVCAnRGVzY3JpcHRpb24gb2YgdGhlIHR5cGUgb2YgZWFjaCBoZXJpdGFnZSBhc3NldCcgCik=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_

Como o Community não suporta table constraints como mostrado acima, será aplicada apenas as limitações de cada coluna

In [0]:
%sql
CREATE OR REPLACE TABLE eleicao_gold.tipo_bem ( 
     CD_TIPO_BEM_CANDIDATO BIGINT NOT NULL COMMENT 'Unique identifier for each heritage asset kind of the candidates' 
    ,DS_TIPO_BEM_CANDIDATO VARCHAR(200) COMMENT 'Description of the type of each heritage asset' 
);

In [0]:
#Tabela relacional tipo de bem (Gold) 
df.createOrReplaceTempView('tipo_bem')

permanent_table_name = "eleicao_gold.tipo_bem"

schema = 'eleicao_gold'

df.write.format("delta").option("mergeSchema", "true").saveAsTable(permanent_table_name,mode='append')

In [0]:
%sql

SELECT 
*
FROM 
  eleicao_gold.tipo_bem

CD_TIPO_BEM_CANDIDATO,DS_TIPO_BEM_CANDIDATO
1,Prédio residencial
2,Prédio comercial
3,Galpão
11,Apartamento
12,Casa
13,Terreno
14,Terra nua
15,Sala ou conjunto
16,Construção
17,Benfeitorias


%md
**Tabela "Candidatos" descritiva particionada por ano**:

In [0]:
#df para tabela relacional relacional candidato
df = sqlContext.sql("""
    SELECT
         l_tb.SQ_CANDIDATO
        ,l_tb.NM_CANDIDATO
        ,l_tb.NM_URNA_CANDIDATO
        ,l_tb.ANO_ELEICAO
        ,l_tb.TP_ABRANGENCIA
        ,l_tb.DT_ELEICAO
        ,l_tb.NR_CPF_CANDIDATO
        ,l_tb.NR_TITULO_ELEITORAL_CANDIDATO
        ,l_tb.DS_CARGO
        ,l_tb.DS_SITUACAO_CANDIDATURA
        ,l_tb.SG_PARTIDO
        ,l_tb.NM_PARTIDO
        ,l_tb.SG_UF_NASCIMENTO
        ,l_tb.DT_NASCIMENTO
        ,l_tb.DS_GENERO
        ,l_tb.DS_COR_RACA
        ,l_tb.DS_GRAU_INSTRUCAO
        ,l_tb.DS_ESTADO_CIVIL
        ,l_tb.DS_OCUPACAO
        ,l_tb.SG_UF
        ,l_tb.NM_UE
        ,l_tb.NR_TURNO
        ,l_tb.DS_SIT_TOT_TURNO
        ,COALESCE(VR_BEM_CANDIDATO,0) AS VR_BEM_CANDIDATO
    FROM 
        eleicao_silver.candidatos AS l_tb
    LEFT JOIN 
        eleicao_silver.bens_candidatos AS r_tb
        ON 
            l_tb.SQ_CANDIDATO = r_tb.SQ_CANDIDATO
            AND l_tb.ANO_ELEICAO = r_tb.ANO_ELEICAO
    """ )
df.limit(10).display()

SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,ANO_ELEICAO,TP_ABRANGENCIA,DT_ELEICAO,NR_CPF_CANDIDATO,NR_TITULO_ELEITORAL_CANDIDATO,DS_CARGO,DS_SITUACAO_CANDIDATURA,SG_PARTIDO,NM_PARTIDO,SG_UF_NASCIMENTO,DT_NASCIMENTO,DS_GENERO,DS_COR_RACA,DS_GRAU_INSTRUCAO,DS_ESTADO_CIVIL,DS_OCUPACAO,SG_UF,NM_UE,NR_TURNO,DS_SIT_TOT_TURNO,VR_BEM_CANDIDATO
250001265966,JOÃO BATISTA DOS SANTOS,JOÃO ZELADOR,2020,MUNICIPAL,2020-11-15,13534893883,292412220132,VEREADOR,INAPTO,DC,DEMOCRACIA CRISTÃ,SP,1970-03-05,MASCULINO,PARDA,ENSINO MÉDIO INCOMPLETO,CASADO(A),GERENTE,SP,SÃO PAULO,1,NÃO ELEITO,0.0
250001268518,SILVIO ROBERTO VOCE,SILVIO VOCE,2020,MUNICIPAL,2020-11-15,90421213868,97094760159,VEREADOR,APTO,PRTB,PARTIDO RENOVADOR TRABALHISTA BRASILEIRO,SP,1953-04-15,MASCULINO,BRANCA,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),COMISSÁRIO DE BORDO,SP,SÃO PAULO,1,NÃO ELEITO,850000.0
250000925963,ANDRESSA MACIEL DO NASCIMENTO,DEDE,2020,MUNICIPAL,2020-11-15,37174723802,344907730124,VEREADOR,INAPTO,PSC,PARTIDO SOCIAL CRISTÃO,SP,1988-11-20,FEMININO,PARDA,SUPERIOR INCOMPLETO,CASADO(A),OUTROS,SP,SÃO PAULO,1,NÃO ELEITO,0.0
160001090215,EDEMAR RIBEIRO BORGES,NICO DA MARACANÃ,2020,MUNICIPAL,2020-11-15,62730614915,4079340655,VEREADOR,APTO,PRTB,PARTIDO RENOVADOR TRABALHISTA BRASILEIRO,PR,1965-01-05,MASCULINO,PARDA,ENSINO FUNDAMENTAL INCOMPLETO,SOLTEIRO(A),MECÂNICO DE MANUTENÇÃO,PR,TUNAS DO PARANÁ,1,NÃO ELEITO,80000.0
250001100745,ROSE LEILA RODASLI CHUERE,ROSE LEILA,2020,MUNICIPAL,2020-11-15,53450779834,156241700167,VEREADOR,INAPTO,PMB,PARTIDO DA MULHER BRASILEIRA,SP,1954-01-11,FEMININO,BRANCA,SUPERIOR COMPLETO,SOLTEIRO(A),APOSENTADO (EXCETO SERVIDOR PÚBLICO),SP,SÃO PAULO,1,NÃO ELEITO,4000000.0
250000984468,JOSÉ AMILTON CORREA,BIKUIRA,2020,MUNICIPAL,2020-11-15,26595781840,180429580124,VEREADOR,APTO,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,PR,1972-08-17,MASCULINO,BRANCA,ENSINO MÉDIO COMPLETO,CASADO(A),TRABALHADOR METALÚRGICO E SIDERÚRGICO,SP,SANTA BRANCA,1,SUPLENTE,33000.0
160000869858,JOSÉ PINTO DE OLIVEIRA,ZÉ DE OLIVEIRA DO GÁS,2020,MUNICIPAL,2020-11-15,10132848953,34885470612,VEREADOR,APTO,PSL,PARTIDO SOCIAL LIBERAL,MG,1951-06-25,MASCULINO,BRANCA,LÊ E ESCREVE,CASADO(A),COMERCIANTE,PR,FAXINAL,1,SUPLENTE,253535.9
130000888282,EVERALDO MACEDO DOS SANTOS,EVERALDO DE ZÉ RODOVIÁRIA,2020,MUNICIPAL,2020-11-15,6747192630,146298280248,VEREADOR,APTO,PTB,PARTIDO TRABALHISTA BRASILEIRO,MG,1984-08-14,MASCULINO,PRETA,ENSINO FUNDAMENTAL COMPLETO,CASADO(A),OUTROS,MG,MONTALVÂNIA,1,SUPLENTE,228000.0
250001123247,ADILSON ADRIANO SALES DE SOUZA CARVALHO AMADEU,DR ADILSON CARVALHO,2020,MUNICIPAL,2020-11-15,28254810842,251049250167,VEREADOR,INAPTO,PSB,PARTIDO SOCIALISTA BRASILEIRO,SP,1974-10-02,MASCULINO,BRANCA,SUPERIOR COMPLETO,SOLTEIRO(A),ADVOGADO,SP,SÃO PAULO,1,NÃO ELEITO,0.0
160000939823,JOAQUIM RODRIGUES NOVO,JOAQUIM RODRIGUES,2020,MUNICIPAL,2020-11-15,59857307949,25950160698,VEREADOR,APTO,PL,PARTIDO LIBERAL,PR,1967-09-12,MASCULINO,BRANCA,ENSINO MÉDIO COMPLETO,CASADO(A),AGRICULTOR,PR,FÊNIX,1,ELEITO POR QP,322633.93


In [0]:
%sql

CREATE OR REPLACE TABLE eleicao_gold.candidatos ( 
     SQ_CANDIDATO BIGINT NOT NULL COMMENT 'Unique identifier for each candidate per year'
    ,NM_CANDIDATO VARCHAR(200) COMMENT 'Name of the candidate'
    ,NM_URNA_CANDIDATO  VARCHAR(200) COMMENT 'Name of the candidate that apperars in ballot box'
    ,ANO_ELEICAO BIGINT NOT NULL COMMENT 'Year of the election'
    ,TP_ABRANGENCIA VARCHAR(100) COMMENT 'Type of election (Municipal or Presidencial)'
    ,DT_ELEICAO DATE COMMENT 'Date of the election, for some can be either the first or the second round'
    ,NR_CPF_CANDIDATO BIGINT COMMENT 'Identity of the candidate'
    ,NR_TITULO_ELEITORAL_CANDIDATO BIGINT COMMENT 'Electoral Identity of the candidate'
    ,DS_CARGO VARCHAR(100) COMMENT 'Position of dispute'
    ,DS_SITUACAO_CANDIDATURA VARCHAR(100) COMMENT 'Situation of the candidature (APTO, CADASTRADO, INAPTO)'
    ,SG_PARTIDO VARCHAR(50) COMMENT 'Abreviation of the name of the political party'
    ,NM_PARTIDO VARCHAR(100) COMMENT 'Name of the political party'
    ,SG_UF_NASCIMENTO VARCHAR(50) COMMENT 'Abreviation of the name of the state of bitht of the candidate'
    ,DT_NASCIMENTO DATE COMMENT 'Date of birth of the candidate'
    ,DS_GENERO VARCHAR(100) COMMENT 'Gender of the candidate (MASCULINO, FEMININO, NÃO DIVULGÁVEL)'
    ,DS_COR_RACA VARCHAR(100) COMMENT 'Etinity Race of the candidate (AMARELA, BRANCA, INDÍGENA, NÃO DIVULGÁVEL, PARDA, PRETA)'
    ,DS_GRAU_INSTRUCAO VARCHAR(100) COMMENT 'Education Degree of the candidate (ENSINO FUNDAMENTAL COMPLETO, ENSINO FUNDAMENTAL INCOMPLETO, ENSINO MÉDIO COMPLETO, ENSINO MÉDIO INCOMPLETO, LÊ E ESCREVE, NÃO DIVULGÁVEL, SUPERIOR  
        COMPLETO, SUPERIOR INCOMPLETO)'
    ,DS_ESTADO_CIVIL  VARCHAR(100) COMMENT 'Marital Status of the candidate (CASADO(A), DIVORCIADO(A), NÃO DIVULGÁVEL, SEPARADO(A) JUDICIALMENTE, SOLTEIRO(A), VIÚVO(A))'
    ,DS_OCUPACAO VARCHAR(200) COMMENT 'Occupation of the candidate'
    ,SG_UF VARCHAR(200) COMMENT 'State of dispute'
    ,NM_UE VARCHAR(200) COMMENT 'District of dispute'
    ,NR_TURNO BIGINT COMMENT 'Last round of dispute'
    ,DS_SIT_TOT_TURNO  VARCHAR(200) COMMENT 'Status of the electoral dispute'
    ,VR_BEM_CANDIDATO DOUBLE COMMENT 'Total amount owned of the candidate'
);

In [0]:
#Tabela relacional candidatos (Gold) 
df.createOrReplaceTempView('candidatos')

permanent_table_name = "eleicao_gold.candidatos"

schema = 'eleicao_gold'

df.write.format("delta").option("mergeSchema", "true").saveAsTable(permanent_table_name,mode='append')

In [0]:
%sql

SELECT 
*
FROM 
  eleicao_gold.candidatos
LIMIT 10

SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,ANO_ELEICAO,TP_ABRANGENCIA,DT_ELEICAO,NR_CPF_CANDIDATO,NR_TITULO_ELEITORAL_CANDIDATO,DS_CARGO,DS_SITUACAO_CANDIDATURA,SG_PARTIDO,NM_PARTIDO,SG_UF_NASCIMENTO,DT_NASCIMENTO,DS_GENERO,DS_COR_RACA,DS_GRAU_INSTRUCAO,DS_ESTADO_CIVIL,DS_OCUPACAO,SG_UF,NM_UE,NR_TURNO,DS_SIT_TOT_TURNO,VR_BEM_CANDIDATO
250001265966,JOÃO BATISTA DOS SANTOS,JOÃO ZELADOR,2020,MUNICIPAL,2020-11-15,13534893883,292412220132,VEREADOR,INAPTO,DC,DEMOCRACIA CRISTÃ,SP,1970-03-05,MASCULINO,PARDA,ENSINO MÉDIO INCOMPLETO,CASADO(A),GERENTE,SP,SÃO PAULO,1,NÃO ELEITO,0.0
250001268518,SILVIO ROBERTO VOCE,SILVIO VOCE,2020,MUNICIPAL,2020-11-15,90421213868,97094760159,VEREADOR,APTO,PRTB,PARTIDO RENOVADOR TRABALHISTA BRASILEIRO,SP,1953-04-15,MASCULINO,BRANCA,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),COMISSÁRIO DE BORDO,SP,SÃO PAULO,1,NÃO ELEITO,850000.0
250000925963,ANDRESSA MACIEL DO NASCIMENTO,DEDE,2020,MUNICIPAL,2020-11-15,37174723802,344907730124,VEREADOR,INAPTO,PSC,PARTIDO SOCIAL CRISTÃO,SP,1988-11-20,FEMININO,PARDA,SUPERIOR INCOMPLETO,CASADO(A),OUTROS,SP,SÃO PAULO,1,NÃO ELEITO,0.0
160001090215,EDEMAR RIBEIRO BORGES,NICO DA MARACANÃ,2020,MUNICIPAL,2020-11-15,62730614915,4079340655,VEREADOR,APTO,PRTB,PARTIDO RENOVADOR TRABALHISTA BRASILEIRO,PR,1965-01-05,MASCULINO,PARDA,ENSINO FUNDAMENTAL INCOMPLETO,SOLTEIRO(A),MECÂNICO DE MANUTENÇÃO,PR,TUNAS DO PARANÁ,1,NÃO ELEITO,80000.0
250001100745,ROSE LEILA RODASLI CHUERE,ROSE LEILA,2020,MUNICIPAL,2020-11-15,53450779834,156241700167,VEREADOR,INAPTO,PMB,PARTIDO DA MULHER BRASILEIRA,SP,1954-01-11,FEMININO,BRANCA,SUPERIOR COMPLETO,SOLTEIRO(A),APOSENTADO (EXCETO SERVIDOR PÚBLICO),SP,SÃO PAULO,1,NÃO ELEITO,4000000.0
250000984468,JOSÉ AMILTON CORREA,BIKUIRA,2020,MUNICIPAL,2020-11-15,26595781840,180429580124,VEREADOR,APTO,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,PR,1972-08-17,MASCULINO,BRANCA,ENSINO MÉDIO COMPLETO,CASADO(A),TRABALHADOR METALÚRGICO E SIDERÚRGICO,SP,SANTA BRANCA,1,SUPLENTE,33000.0
160000869858,JOSÉ PINTO DE OLIVEIRA,ZÉ DE OLIVEIRA DO GÁS,2020,MUNICIPAL,2020-11-15,10132848953,34885470612,VEREADOR,APTO,PSL,PARTIDO SOCIAL LIBERAL,MG,1951-06-25,MASCULINO,BRANCA,LÊ E ESCREVE,CASADO(A),COMERCIANTE,PR,FAXINAL,1,SUPLENTE,253535.9
130000888282,EVERALDO MACEDO DOS SANTOS,EVERALDO DE ZÉ RODOVIÁRIA,2020,MUNICIPAL,2020-11-15,6747192630,146298280248,VEREADOR,APTO,PTB,PARTIDO TRABALHISTA BRASILEIRO,MG,1984-08-14,MASCULINO,PRETA,ENSINO FUNDAMENTAL COMPLETO,CASADO(A),OUTROS,MG,MONTALVÂNIA,1,SUPLENTE,228000.0
250001123247,ADILSON ADRIANO SALES DE SOUZA CARVALHO AMADEU,DR ADILSON CARVALHO,2020,MUNICIPAL,2020-11-15,28254810842,251049250167,VEREADOR,INAPTO,PSB,PARTIDO SOCIALISTA BRASILEIRO,SP,1974-10-02,MASCULINO,BRANCA,SUPERIOR COMPLETO,SOLTEIRO(A),ADVOGADO,SP,SÃO PAULO,1,NÃO ELEITO,0.0
160000939823,JOAQUIM RODRIGUES NOVO,JOAQUIM RODRIGUES,2020,MUNICIPAL,2020-11-15,59857307949,25950160698,VEREADOR,APTO,PL,PARTIDO LIBERAL,PR,1967-09-12,MASCULINO,BRANCA,ENSINO MÉDIO COMPLETO,CASADO(A),AGRICULTOR,PR,FÊNIX,1,ELEITO POR QP,322633.93


In [0]:
%sql
SELECT
  COUNT(*) AS VALIDATION
FROM 
  eleicao_silver.candidatos

VALIDATION
617061


In [0]:
%sql
SELECT
  COUNT(*) AS VALIDATION
FROM 
  eleicao_gold.candidatos

VALIDATION
617061


Como validação as quantidades de linhas das tabelas silver e gold, devem ser as mesmas.

%md
**Tabela "Bens" descritiva particionada por ano e tipo de bem**:

Nessa tabela foram encontrados 50 ocorrências com "SQ_CANDIDATO" nulos, serão desconsideradas nas tabelas gold 

In [0]:
%sql
SELECT
      COUNT(*) contagem
FROM 
    eleicao_silver.bens_candidatos_tipo
WHERE 
   SQ_CANDIDATO IS NULL

contagem
50


In [0]:
#df bens candidatos 
df = sqlContext.sql("""
    SELECT
         SQ_CANDIDATO
        ,ANO_ELEICAO
        ,CD_TIPO_BEM_CANDIDATO
        ,DS_TIPO_BEM_CANDIDATO
        ,VR_BEM_CANDIDATO
    FROM 
        eleicao_silver.bens_candidatos_tipo
    WHERE 
        SQ_CANDIDATO IS NOT NULL
    """ )
df.limit(10).display()

SQ_CANDIDATO,ANO_ELEICAO,CD_TIPO_BEM_CANDIDATO,DS_TIPO_BEM_CANDIDATO,VR_BEM_CANDIDATO
210000773759,2020,45,"Aplicação de renda fixa (CDB, RDB e outros)",2765.67
210000773942,2020,45,"Aplicação de renda fixa (CDB, RDB e outros)",4886.7699999999995
160000924651,2020,13,Terreno,30000.0
130000778579,2020,13,Terreno,30000.0
160001145428,2020,26,Linha telefônica,3909.46
130000963313,2020,32,Quotas ou quinhões de capital,4000.0
160000877803,2020,13,Terreno,25000.0
130000826790,2020,32,Quotas ou quinhões de capital,500000.0
60000984156,2020,45,"Aplicação de renda fixa (CDB, RDB e outros)",11590.14
170000650492,2020,13,Terreno,12000.0


In [0]:
%sql

CREATE OR REPLACE TABLE eleicao_gold.bens_candidatos_tipo ( 
     SQ_CANDIDATO BIGINT NOT NULL COMMENT 'Unique identifier for each candidate per year'
    ,ANO_ELEICAO BIGINT NOT NULL COMMENT 'Year of the election'
    ,CD_TIPO_BEM_CANDIDATO BIGINT COMMENT 'Types of heritage assets declared by the candidate'
    ,DS_TIPO_BEM_CANDIDATO VARCHAR(200) COMMENT 'description of types of heritage assets declared by the candidate'
    ,VR_BEM_CANDIDATO DOUBLE COMMENT 'Total amount owned of the candidate per type of asset'
);

In [0]:
#Tabela relacional bens por tipo (Gold) 
df.createOrReplaceTempView('bens_candidatos_tipo')

permanent_table_name = "eleicao_gold.bens_candidatos_tipo"

schema = 'eleicao_gold'

df.write.format("delta").option("mergeSchema", "true").saveAsTable(permanent_table_name,mode='append')

In [0]:
%sql
SELECT 
    *
FROM 
  eleicao_gold.bens_candidatos_tipo
LIMIT 10

SQ_CANDIDATO,ANO_ELEICAO,CD_TIPO_BEM_CANDIDATO,DS_TIPO_BEM_CANDIDATO,VR_BEM_CANDIDATO
160001144090,2020,13,Terreno,0.0
130000795365,2020,19,Outros bens imóveis,50000.0
50001061515,2020,19,Outros bens imóveis,9000.0
160001204549,2020,13,Terreno,350000.0
130000827854,2020,13,Terreno,50000.0
130000843073,2020,19,Outros bens imóveis,50000.0
60000984399,2020,13,Terreno,25000.0
60001243623,2020,13,Terreno,28000.0
160000782330,2020,45,"Aplicação de renda fixa (CDB, RDB e outros)",13791.05
190000636318,2020,13,Terreno,98000.0


In [0]:
%sql
--validação 
SELECT 
    *
FROM 
    eleicao_gold.candidatos
WHERE 
    NR_CPF_CANDIDATO = 1859237711 --exemplo aleatório


SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,ANO_ELEICAO,TP_ABRANGENCIA,DT_ELEICAO,NR_CPF_CANDIDATO,NR_TITULO_ELEITORAL_CANDIDATO,DS_CARGO,DS_SITUACAO_CANDIDATURA,SG_PARTIDO,NM_PARTIDO,SG_UF_NASCIMENTO,DT_NASCIMENTO,DS_GENERO,DS_COR_RACA,DS_GRAU_INSTRUCAO,DS_ESTADO_CIVIL,DS_OCUPACAO,SG_UF,NM_UE,NR_TURNO,DS_SIT_TOT_TURNO,VR_BEM_CANDIDATO
150000676096,RUY MANUEL CARNEIRO BARBOSA DE AÇA BELCHIOR,RUY CARNEIRO,2020,MUNICIPAL,2020-11-15,1859237711,19327891279,PREFEITO,APTO,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,RJ,1970-08-17,MASCULINO,BRANCA,SUPERIOR COMPLETO,CASADO(A),DEPUTADO,PB,JOÃO PESSOA,1,NÃO ELEITO,649968.67
150000612366,RUY MANUEL CARNEIRO BARBOSA DE AÇA BELCHIOR,RUY CARNEIRO,2018,ESTADUAL,2018-10-07,1859237711,19327891279,DEPUTADO FEDERAL,APTO,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,RJ,1970-08-17,MASCULINO,BRANCA,SUPERIOR COMPLETO,CASADO(A),OUTROS,PB,PARAÍBA,1,ELEITO POR MÉDIA,586284.8200000001
150001612089,RUY MANUEL CARNEIRO BARBOSA DE AÇA BELCHIOR,RUY CARNEIRO,2022,ESTADUAL,2022-10-02,1859237711,19327891279,DEPUTADO FEDERAL,APTO,PSC,PARTIDO SOCIAL CRISTÃO,RJ,1970-08-17,MASCULINO,BRANCA,SUPERIOR COMPLETO,CASADO(A),DEPUTADO,PB,PARAÍBA,1,ELEITO POR MÉDIA,0.0


**Tabela para análise**

Nesta tabela será disponibilizado os valores de evolução patrimonial para a análise a ser realizada.<br> 
A Estrutura dela será:


In [0]:
%sql
CREATE OR REPLACE TABLE eleicao_gold.patrimonio ( 
     NM_CANDIDATO VARCHAR(200) COMMENT 'Name of the candidate in 2022'
    ,NM_URNA_CANDIDATO  VARCHAR(200) COMMENT 'Name of the candidate that apperars in ballot box in 2022'
    ,TP_ABRANGENCIA VARCHAR(100) COMMENT 'Type of election (Municipal or Presidencial) in 2022'
    ,NR_CPF_CANDIDATO BIGINT COMMENT 'Identity of the candidate'
    ,NR_TITULO_ELEITORAL_CANDIDATO BIGINT COMMENT 'Electoral Identity of the candidate'
    ,DS_CARGO VARCHAR(100) COMMENT 'Position of dispute in 2022'
    ,DS_CARGO_2020 VARCHAR(100) COMMENT 'Position of dispute in 2020'
    ,DS_CARGO_2018 VARCHAR(100) COMMENT 'Position of dispute in 2018'
    ,DS_SITUACAO_CANDIDATURA VARCHAR(100) COMMENT 'Situation of the candidature (APTO, CADASTRADO, INAPTO) in 2022'
    ,DS_SITUACAO_CANDIDATURA_2020 VARCHAR(100) COMMENT 'Situation of the candidature (APTO, CADASTRADO, INAPTO) in 2020'
    ,DS_SITUACAO_CANDIDATURA_2018 VARCHAR(100) COMMENT 'Situation of the candidature (APTO, CADASTRADO, INAPTO) in 2018'
    ,DT_ELEICAO DATE COMMENT 'Date of the election, for some can be either the first or the second round'
    ,SG_PARTIDO VARCHAR(50) COMMENT 'Abreviation of the name of the political party in 2022'
    ,NM_PARTIDO VARCHAR(100) COMMENT 'Name of the political party in 2022'
    ,DT_NASCIMENTO DATE COMMENT 'Date of birth of the candidate'
    ,DS_GENERO VARCHAR(100) COMMENT 'Gender of the candidate (MASCULINO, FEMININO, NÃO DIVULGÁVEL) declared in 2022'
    ,DS_COR_RACA VARCHAR(100) COMMENT 'Etinity Race of the candidate (AMARELA, BRANCA, INDÍGENA, NÃO DIVULGÁVEL, PARDA, PRETA) declared in 2022'
    ,DS_GRAU_INSTRUCAO VARCHAR(100) COMMENT 'Education Degree of the candidate (ENSINO FUNDAMENTAL COMPLETO, ENSINO FUNDAMENTAL INCOMPLETO, ENSINO MÉDIO COMPLETO, ENSINO MÉDIO INCOMPLETO, LÊ E ESCREVE, NÃO DIVULGÁVEL, SUPERIOR  
        COMPLETO, SUPERIOR INCOMPLETO) declared in 2022'
    ,DS_ESTADO_CIVIL  VARCHAR(100) COMMENT 'Marital Status of the candidate (CASADO(A), DIVORCIADO(A), NÃO DIVULGÁVEL, SEPARADO(A) JUDICIALMENTE, SOLTEIRO(A), VIÚVO(A)) declared in 2022'
    ,DS_OCUPACAO VARCHAR(200) COMMENT 'Occupation of the candidate declared in 2022'
    ,SG_UF VARCHAR(200) COMMENT 'State of dispute in 2022'
    ,NM_UE VARCHAR(200) COMMENT 'District of dispute in 2022'
    ,DS_SIT_TOT_TURNO  VARCHAR(200) COMMENT 'Status of the electoral dispute in 2022'
    ,DS_SIT_TOT_TURNO_2020  VARCHAR(200) COMMENT 'Status of the electoral dispute in 2020'
    ,DS_SIT_TOT_TURNO_2018  VARCHAR(200) COMMENT 'Status of the electoral dispute in 2018'
    ,VR_BEM_CANDIDATO DOUBLE COMMENT 'Total amount owned of the candidate in 2022'
    ,VR_BEM_CANDIDATO_2020 DOUBLE COMMENT 'Total amount owned of the candidate in 2020'
    ,VR_BEM_CANDIDATO_2018 DOUBLE COMMENT 'Total amount owned of the candidate in 2018'
);

In [0]:
#df para tabela relacional de patrimonio de candidato gold
df = sqlContext.sql("""
  WITH 
    candidatos_2022 AS (
      SELECT
           l_tb.NM_CANDIDATO
          ,l_tb.NM_URNA_CANDIDATO
          ,l_tb.TP_ABRANGENCIA
          ,l_tb.NR_CPF_CANDIDATO
          ,l_tb.NR_TITULO_ELEITORAL_CANDIDATO
          ,l_tb.SG_UF_NASCIMENTO
          ,l_tb.DT_NASCIMENTO
          ,l_tb.DS_CARGO
          ,l_tb.DS_SITUACAO_CANDIDATURA
          ,l_tb.DT_ELEICAO
          ,l_tb.SG_PARTIDO
          ,l_tb.NM_PARTIDO
          ,l_tb.DS_GENERO
          ,l_tb.DS_COR_RACA
          ,l_tb.DS_GRAU_INSTRUCAO
          ,l_tb.DS_ESTADO_CIVIL
          ,l_tb.DS_OCUPACAO
          ,l_tb.SG_UF
          ,l_tb.NM_UE
          ,l_tb.DS_SIT_TOT_TURNO
          ,VR_BEM_CANDIDATO
      FROM 
          eleicao_silver.candidatos AS l_tb
      LEFT JOIN 
          eleicao_silver.bens_candidatos AS r_tb
          ON 
            l_tb.SQ_CANDIDATO = r_tb.SQ_CANDIDATO
            AND l_tb.ANO_ELEICAO = r_tb.ANO_ELEICAO
      WHERE 
        l_tb.ANO_ELEICAO = 2022
    )

    ,candidatos_2020 AS (
      SELECT
           NM_CANDIDATO
          ,l_tb.NR_CPF_CANDIDATO
          ,l_tb.NR_TITULO_ELEITORAL_CANDIDATO
          ,l_tb.SG_UF_NASCIMENTO
          ,l_tb.DT_NASCIMENTO
          ,l_tb.DS_CARGO AS DS_CARGO_2020
          ,l_tb.DS_SITUACAO_CANDIDATURA AS DS_SITUACAO_CANDIDATURA_2020
          ,l_tb.DS_SIT_TOT_TURNO AS DS_SIT_TOT_TURNO_2020
          ,VR_BEM_CANDIDATO AS VR_BEM_CANDIDATO_2020
      FROM 
          eleicao_silver.candidatos AS l_tb
      LEFT JOIN 
          eleicao_silver.bens_candidatos AS r_tb
          ON 
            l_tb.SQ_CANDIDATO = r_tb.SQ_CANDIDATO
            AND l_tb.ANO_ELEICAO = r_tb.ANO_ELEICAO
      WHERE 
        l_tb.ANO_ELEICAO = 2020
    )

    ,candidatos_2018 AS (
        SELECT
           NM_CANDIDATO
          ,l_tb.NR_CPF_CANDIDATO
          ,l_tb.NR_TITULO_ELEITORAL_CANDIDATO
          ,l_tb.SG_UF_NASCIMENTO
          ,l_tb.DT_NASCIMENTO
          ,l_tb.DS_CARGO AS DS_CARGO_2018
          ,l_tb.DS_SITUACAO_CANDIDATURA AS DS_SITUACAO_CANDIDATURA_2018
          ,l_tb.DS_SIT_TOT_TURNO AS DS_SIT_TOT_TURNO_2018
          ,VR_BEM_CANDIDATO AS VR_BEM_CANDIDATO_2018
      FROM 
          eleicao_silver.candidatos AS l_tb
      LEFT JOIN 
          eleicao_silver.bens_candidatos AS r_tb
          ON 
            l_tb.SQ_CANDIDATO = r_tb.SQ_CANDIDATO
            AND l_tb.ANO_ELEICAO = r_tb.ANO_ELEICAO
      WHERE 
        l_tb.ANO_ELEICAO = 2018
    )
    SELECT
         C_2022.NM_CANDIDATO
        ,NM_URNA_CANDIDATO
        ,TP_ABRANGENCIA
        ,C_2022.NR_CPF_CANDIDATO
        ,C_2022.NR_TITULO_ELEITORAL_CANDIDATO
        ,DS_CARGO
        ,DS_CARGO_2020
        ,DS_CARGO_2018
        ,DS_SITUACAO_CANDIDATURA
        ,DS_SITUACAO_CANDIDATURA_2020
        ,DS_SITUACAO_CANDIDATURA_2018
        ,DT_ELEICAO
        ,SG_PARTIDO
        ,NM_PARTIDO
        ,C_2022.DT_NASCIMENTO
        ,DS_GENERO
        ,DS_COR_RACA
        ,DS_GRAU_INSTRUCAO
        ,DS_ESTADO_CIVIL
        ,DS_OCUPACAO
        ,SG_UF
        ,NM_UE
        ,DS_SIT_TOT_TURNO
        ,DS_SIT_TOT_TURNO_2020
        ,DS_SIT_TOT_TURNO_2018
        ,VR_BEM_CANDIDATO
        ,VR_BEM_CANDIDATO_2020
        ,VR_BEM_CANDIDATO_2018
    FROM 
        candidatos_2022 AS C_2022
    LEFT JOIN 
        candidatos_2020 AS C_2020
        ON C_2022.NR_CPF_CANDIDATO = C_2020.NR_CPF_CANDIDATO --join via cpf
        OR (C_2022.NM_CANDIDATO = C_2020.NM_CANDIDATO AND C_2022.SG_UF_NASCIMENTO = C_2020.SG_UF_NASCIMENTO AND C_2022.DT_NASCIMENTO = C_2020.DT_NASCIMENTO) --ou via nome com estado e data de nascimento
    LEFT JOIN 
        candidatos_2018 AS C_2018
        ON C_2022.NR_CPF_CANDIDATO = C_2018.NR_CPF_CANDIDATO--join via cpf
        OR (C_2022.NM_CANDIDATO = C_2018.NM_CANDIDATO AND C_2022.SG_UF_NASCIMENTO = C_2018.SG_UF_NASCIMENTO AND C_2022.DT_NASCIMENTO = C_2018.DT_NASCIMENTO) --ou via nome com estado e data de nascimento
            
    """ )
df.limit(10).display()

NM_CANDIDATO,NM_URNA_CANDIDATO,TP_ABRANGENCIA,NR_CPF_CANDIDATO,NR_TITULO_ELEITORAL_CANDIDATO,DS_CARGO,DS_CARGO_2020,DS_CARGO_2018,DS_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA_2020,DS_SITUACAO_CANDIDATURA_2018,DT_ELEICAO,SG_PARTIDO,NM_PARTIDO,DT_NASCIMENTO,DS_GENERO,DS_COR_RACA,DS_GRAU_INSTRUCAO,DS_ESTADO_CIVIL,DS_OCUPACAO,SG_UF,NM_UE,DS_SIT_TOT_TURNO,DS_SIT_TOT_TURNO_2020,DS_SIT_TOT_TURNO_2018,VR_BEM_CANDIDATO,VR_BEM_CANDIDATO_2020,VR_BEM_CANDIDATO_2018
RAVENNA DE CASTRO LIMA AZEVEDO,RAVENNA CASTRO,ESTADUAL,854325395,29874391546,GOVERNADOR,VEREADOR,DEPUTADO ESTADUAL,APTO,APTO,APTO,2022-10-02,PMN,PARTIDO DA MOBILIZAÇÃO NACIONAL,1986-05-15,FEMININO,PARDA,SUPERIOR COMPLETO,DIVORCIADO(A),ADVOGADO,PI,PIAUÍ,NÃO ELEITO,NÃO ELEITO,SUPLENTE,null,90019.07,15000.0
SORAYA VIEIRA THRONICKE,SORAYA THRONICKE,FEDERAL,60838965172,13250921929,PRESIDENTE,null,SENADOR,APTO,null,APTO,2022-10-02,UNIÃO,UNIÃO BRASIL,1973-06-01,FEMININO,BRANCA,SUPERIOR COMPLETO,CASADO(A),SENADOR,BR,BRASIL,NÃO ELEITO,null,ELEITO,null,null,10000.0
JAIME DOMINGUES NUNES,JAIME NUNES,ESTADUAL,14673835204,285072526,GOVERNADOR,null,VICE-GOVERNADOR,APTO,null,APTO,2022-10-02,PSD,PARTIDO SOCIAL DEMOCRÁTICO,1959-07-29,MASCULINO,BRANCA,SUPERIOR COMPLETO,CASADO(A),EMPRESÁRIO,AP,AMAPÁ,NÃO ELEITO,null,ELEITO,3.577429492E7,null,2.1885903150000006E7
JOÃO INÁCIO RIBEIRO ROMA NETO,JOÃO ROMA,ESTADUAL,81968442472,37641250850,GOVERNADOR,null,DEPUTADO FEDERAL,APTO,null,APTO,2022-10-02,PL,PARTIDO LIBERAL,1972-11-17,MASCULINO,BRANCA,SUPERIOR COMPLETO,CASADO(A),PRODUTOR AGROPECUÁRIO,BA,BAHIA,NÃO ELEITO,null,ELEITO POR QP,5561182.6099999985,null,4494372.77
GILVAM PINHEIRO BORGES,GILVAM BORGES,ESTADUAL,11907894268,346522577,GOVERNADOR,null,SENADOR,APTO,null,APTO,2022-10-02,MDB,MOVIMENTO DEMOCRÁTICO BRASILEIRO,1958-08-01,MASCULINO,PARDA,SUPERIOR COMPLETO,SOLTEIRO(A),SOCIÓLOGO,AP,AMAPÁ,NÃO ELEITO,null,NÃO ELEITO,null,null,408400.0
JOAO BARBOSA BRAVO,PROFESSOR BRAVO,FEDERAL,3532046768,45076570302,VICE-PRESIDENTE,null,null,APTO,null,null,2022-10-02,DC,DEMOCRACIA CRISTÃ,1947-06-27,MASCULINO,BRANCA,SUPERIOR COMPLETO,CASADO(A),ECONOMISTA,BR,BRASIL,NÃO ELEITO,null,null,null,null,null
FERNANDO GALVAO MOURA,FERNANDO GALVÃO,ESTADUAL,10890650861,194255440116,DEPUTADO ESTADUAL,null,null,APTO,null,null,2022-10-02,UNIÃO,UNIÃO BRASIL,1972-12-11,MASCULINO,BRANCA,SUPERIOR COMPLETO,CASADO(A),ADVOGADO,SP,SÃO PAULO,SUPLENTE,null,null,null,null,null
GESIEL DE SOUZA OLIVEIRA,GESIEL DE OLIVEIRA,ESTADUAL,61341509249,2462812585,GOVERNADOR,null,null,APTO,null,null,2022-10-02,PRTB,PARTIDO RENOVADOR TRABALHISTA BRASILEIRO,1978-02-03,MASCULINO,BRANCA,SUPERIOR COMPLETO,CASADO(A),OUTROS,AP,AMAPÁ,NÃO ELEITO,null,null,160000.0,null,null
ANTONIO ALVES DA SILVA JUNIOR,ANTONIO ALVES,FEDERAL,4551706493,56283400876,VICE-PRESIDENTE,null,null,APTO,null,null,2022-10-02,PCB,PARTIDO COMUNISTA BRASILEIRO,1978-10-07,MASCULINO,PRETA,SUPERIOR COMPLETO,SOLTEIRO(A),JORNALISTA E REDATOR,BR,BRASIL,NÃO ELEITO,null,null,null,null,null
RAIMUNDO SALES DA SILVA,BIGODE,ESTADUAL,5135834249,837222496,DEPUTADO ESTADUAL,null,null,APTO,null,null,2022-10-02,PP,PROGRESSISTAS,1955-04-10,MASCULINO,PARDA,ENSINO FUNDAMENTAL COMPLETO,DIVORCIADO(A),PESCADOR,AC,ACRE,SUPLENTE,null,null,null,null,null


In [0]:
#Tabela relacional bens por tipo (Gold) 
df.createOrReplaceTempView('patrimonio')

permanent_table_name = "eleicao_gold.patrimonio"

schema = 'eleicao_gold'

df.write.format("delta").option("mergeSchema", "true").saveAsTable(permanent_table_name,mode='append')

In [0]:
%sql
SELECT
*
FROM 
  eleicao_gold.patrimonio
LIMIT 10

NM_CANDIDATO,NM_URNA_CANDIDATO,TP_ABRANGENCIA,NR_CPF_CANDIDATO,NR_TITULO_ELEITORAL_CANDIDATO,DS_CARGO,DS_CARGO_2020,DS_CARGO_2018,DS_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA_2020,DS_SITUACAO_CANDIDATURA_2018,DT_ELEICAO,SG_PARTIDO,NM_PARTIDO,DT_NASCIMENTO,DS_GENERO,DS_COR_RACA,DS_GRAU_INSTRUCAO,DS_ESTADO_CIVIL,DS_OCUPACAO,SG_UF,NM_UE,DS_SIT_TOT_TURNO,DS_SIT_TOT_TURNO_2020,DS_SIT_TOT_TURNO_2018,VR_BEM_CANDIDATO,VR_BEM_CANDIDATO_2020,VR_BEM_CANDIDATO_2018
JAIRO SANTANA DA SILVA,JAIRO DE GLÓRIA,ESTADUAL,80208053549,12328342100,DEPUTADO ESTADUAL,PREFEITO,DEPUTADO ESTADUAL,APTO,APTO,APTO,2022-10-02,PP,PROGRESSISTAS,1979-06-22,MASCULINO,PARDA,SUPERIOR COMPLETO,DIVORCIADO(A),CONTADOR,SE,SERGIPE,SUPLENTE,NÃO ELEITO,SUPLENTE,null,null,null
GELSON FERRAZ DE MEDEIROS,GELSON FERRAZ,ESTADUAL,32983506720,48856060710,1º SUPLENTE,VEREADOR,DEPUTADO ESTADUAL,APTO,APTO,APTO,2022-10-02,AVANTE,AVANTE,1953-01-18,MASCULINO,PARDA,ENSINO MÉDIO COMPLETO,CASADO(A),LOCUTOR E COMENTARISTA DE RÁDIO E TELEVISÃO E RADIALISTA,CE,CEARÁ,NÃO ELEITO,SUPLENTE,SUPLENTE,501117.38,116560.32,11539.82
JAMIR CALILI RIBEIRO,JAMIR CALILI RIBEIRO,ESTADUAL,1304762696,134168860213,DEPUTADO ESTADUAL,VEREADOR,DEPUTADO ESTADUAL,APTO,APTO,APTO,2022-10-02,PP,PROGRESSISTAS,1982-10-21,MASCULINO,BRANCA,SUPERIOR COMPLETO,CASADO(A),PROFESSOR DE ENSINO SUPERIOR,MG,MINAS GERAIS,SUPLENTE,ELEITO POR QP,SUPLENTE,null,1415000.0,1789060.0
MANOEL FRANCISCO LIMA DE SOUZA,MANOEL IZO,ESTADUAL,47815337287,2387752410,DEPUTADO ESTADUAL,VEREADOR,DEPUTADO ESTADUAL,APTO,APTO,APTO,2022-10-02,DC,DEMOCRACIA CRISTÃ,1971-10-28,MASCULINO,PRETA,ENSINO MÉDIO INCOMPLETO,CASADO(A),OUTROS,AC,ACRE,NÃO ELEITO,NÃO ELEITO,NÃO ELEITO,null,null,23000.0
JOSÉ ÔNIX GOMES DA SILVA,JOSÉ ÔNIX RÁDIO RIO BONITO,ESTADUAL,3220945775,97887740396,DEPUTADO FEDERAL,VEREADOR,DEPUTADO ESTADUAL,APTO,APTO,APTO,2022-10-02,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,1977-09-18,MASCULINO,PRETA,ENSINO MÉDIO COMPLETO,DIVORCIADO(A),VIGILANTE,RJ,RIO DE JANEIRO,NÃO ELEITO,SUPLENTE,SUPLENTE,null,null,null
CESAR ANTONIO VALDUGA,VALDUGA,ESTADUAL,42570778915,16092430906,DEPUTADO ESTADUAL,VEREADOR,DEPUTADO ESTADUAL,APTO,APTO,APTO,2022-10-02,PC do B,PARTIDO COMUNISTA DO BRASIL,1961-05-20,MASCULINO,BRANCA,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),BANCÁRIO E ECONOMIÁRIO,SC,SANTA CATARINA,SUPLENTE,ELEITO POR QP,SUPLENTE,null,2957746.77,526788.1900000001
MARIANA ROSSETO SANTOS,PROFESSORA MARIANA,ESTADUAL,10360020801,193058900116,DEPUTADO ESTADUAL,VEREADOR,DEPUTADO FEDERAL,APTO,APTO,APTO,2022-10-02,PT,PARTIDO DOS TRABALHADORES,1973-07-13,FEMININO,BRANCA,SUPERIOR COMPLETO,SOLTEIRO(A),PROFESSOR DE ENSINO MÉDIO,SP,SÃO PAULO,SUPLENTE,NÃO ELEITO,SUPLENTE,null,78413.98000000001,62000.0
LUIZ EDUARDO CARNEIRO DA SILVA DE SOUZA LIMA,LUIZ LIMA,ESTADUAL,7616393730,100242820302,DEPUTADO FEDERAL,PREFEITO,DEPUTADO FEDERAL,APTO,APTO,APTO,2022-10-02,PL,PARTIDO LIBERAL,1977-12-10,MASCULINO,BRANCA,SUPERIOR COMPLETO,SOLTEIRO(A),DEPUTADO,RJ,RIO DE JANEIRO,ELEITO POR QP,NÃO ELEITO,ELEITO POR QP,null,1537681.74,1749011.3500000003
CRISTINA ALVES DE GODOY,CRIS GODOY,ESTADUAL,13563540845,188441480141,DEPUTADO ESTADUAL,VEREADOR,DEPUTADO ESTADUAL,APTO,APTO,APTO,2022-10-02,PTB,PARTIDO TRABALHISTA BRASILEIRO,1971-01-31,FEMININO,BRANCA,SUPERIOR COMPLETO,DIVORCIADO(A),PSICÓLOGO,SP,SÃO PAULO,NÃO ELEITO,NÃO ELEITO,SUPLENTE,null,51074.26,117036.30000000002
DELBIO CAMARGO TERUEL,DELBIO TERUEL,ESTADUAL,6830290880,110834780108,DEPUTADO FEDERAL,VEREADOR,DEPUTADO ESTADUAL,APTO,APTO,APTO,2022-10-02,UNIÃO,UNIÃO BRASIL,1966-06-29,MASCULINO,BRANCA,SUPERIOR COMPLETO,DIVORCIADO(A),VEREADOR,SP,SÃO PAULO,SUPLENTE,ELEITO POR MÉDIA,SUPLENTE,null,922084.1100000001,960770.9800000001


Como visto, para garantir o máximo possível a assertividade do join, foram utilizados os campos de "SG_UF_NASCIMENTO" e "DT_NASCIMENTO".

------------------------------------------

# Análise:
A seguir apenas respostas rápidas para perguntas feitas:


OBS: Essa análise usa dados do TSE sobre os bens _declarados_ dos candidatos, será considerada apenas a evolução de candidatos recorrentes, isto é, pessoas que se candidataram em duas eleições. Isso pode diminuir enormemente nossa base e a confiabilidade, entretando ressalto que esse estudo é meramente para fins de projeto MVP para conclusão do módulo sobre **Engenharia de Dados** e para satisfação da curiosidade do autor, ressalto que o intuito aqui não é qualquer vínculo de cunho político ou partidário, apenas a satisfação de um aprofundamento sob dados aplamente divulgados pelos orgãos competentes, ademais segue metodologia da análise:

### Metodologia
Avaliar evolução patrimonial _declarada_ é algo bem complexo, vários fatores externos podem estar ligadas a diminuição ou aumento que não sejam o servidorismo, como "muleta" técnica usarei informações como quantidade de candidatos em cada particionamento, a média de patrimônio declarada e a quantidade de declarações realizadas em cada ano por tipo.
- As comparações serão feitas entre doi anos, lembrando que olharemos apenas candidatos recorrentes, exemplo: qual a evolução de patrimonio entre candidatos recorrentes entre os anos de 2020 e 2022...
  

### Pergunta Principal:
#### Qual a evolução patrimonial dos candidatos recorrentes da eleição de 2022?

### Perguntas de Desdobramento:
1. Qual média patrimonial dos candidatos (por cargo concorrido)?
2. Candidatos eleitos tem variação diferente de patrimônio?
3. A variação é diferente por partido?
4. A variação é diferente entre gênero?
5. A variação é diferente por idade?

In [0]:
%sql
SELECT
   COUNT(DISTINCT IF(ANO_ELEICAO = 2022,SQ_CANDIDATO,NULL)) AS QTD_CANDIDATOS_DECLARACOES_2022
  ,SUM(IF(ANO_ELEICAO = 2022,VR_BEM_CANDIDATO,NULL)) AS VR_TOTAL_DECLARADO_2022
  ,COUNT(DISTINCT IF(ANO_ELEICAO = 2020,SQ_CANDIDATO,NULL)) AS QTD_CANDIDATOS_DECLARACOES_2020
  ,SUM(IF(ANO_ELEICAO = 2020,VR_BEM_CANDIDATO,NULL)) AS VR_TOTAL_DECLARADO_2020
  ,COUNT(DISTINCT IF(ANO_ELEICAO = 2018,SQ_CANDIDATO,NULL)) AS QTD_CANDIDATOS_DECLARACOES_2018
  ,SUM(IF(ANO_ELEICAO = 2018,VR_BEM_CANDIDATO,NULL)) AS VR_TOTAL_DECLARADO_2018
FROM 
  eleicao_silver.bens_candidatos

QTD_CANDIDATOS_DECLARACOES_2022,VR_TOTAL_DECLARADO_2022,QTD_CANDIDATOS_DECLARACOES_2020,VR_TOTAL_DECLARADO_2020,QTD_CANDIDATOS_DECLARACOES_2018,VR_TOTAL_DECLARADO_2018
4655,2.299662642503998E10,340935,1.150809522064301E11,17646,2.422924231855999E10


Repare que há bem menos candidatos com declarações no ano de 2022, mais que 4x menos comparado a 2018 que foram eleições do mesmo ambito, segundo a documentação fornecida podem ter havido em 2022 mais pedidos de sigilo/não divulgação ou simplesmente menos declarações.

In [0]:
%sql
SELECT
   AVG(IF(ANO_ELEICAO = 2022,VR_BEM_CANDIDATO,NULL)) AS MEDIA_DECLARACOES_2022
  ,AVG(IF(ANO_ELEICAO = 2020,VR_BEM_CANDIDATO,NULL)) AS MEDIA_DECLARACOES_2020
  ,AVG(IF(ANO_ELEICAO = 2018,VR_BEM_CANDIDATO,NULL)) AS MEDIA_DECLARACOES_2018
FROM 
  eleicao_silver.bens_candidatos

MEDIA_DECLARACOES_2022,MEDIA_DECLARACOES_2020,MEDIA_DECLARACOES_2018
4979780.516465998,339173.68274033477,1373072.7824186778


A média patrimonial declarada por candidato teve uma variação alta, tanto de 2018 para 2022, quanto entre 2020 e 2022 (essa última eleições de ambitos diferentes).

Para a variação usarei a fórmula simples de diferença: (Valor atual/Valo anterior) - 1

In [0]:
%sql
--Primeira comparação entre 2018 e 2022
SELECT
   COUNT(DISTINCT NR_CPF_CANDIDATO)
  ,AVG(VR_BEM_CANDIDATO) AS MEDIA_BENS_2022
  ,AVG(VR_BEM_CANDIDATO_2018) AS MEDIA_BENS_2018
  ,SUM(VR_BEM_CANDIDATO) AS VR_BEM_CANDIDATO_2022
  ,SUM(VR_BEM_CANDIDATO_2018) AS VR_BEM_CANDIDATO_2018
  ,SUM(VR_BEM_CANDIDATO) / SUM(VR_BEM_CANDIDATO_2018) AS VARIACAO_22_18
  ,SUM(VR_BEM_CANDIDATO) / SUM(VR_BEM_CANDIDATO_2018) - 1  AS PERC_VARIACAO_22_18
FROM 
  eleicao_gold.patrimonio
WHERE 
  DS_CARGO IS NOT NULL AND DS_CARGO_2018 IS NOT NULL AND VR_BEM_CANDIDATO IS NOT NULL AND VR_BEM_CANDIDATO_2018 IS NOT NULL

count(DISTINCT NR_CPF_CANDIDATO),MEDIA_BENS_2022,MEDIA_BENS_2018,VR_BEM_CANDIDATO_2022,VR_BEM_CANDIDATO_2018,VARIACAO_22_18,PERC_VARIACAO_22_18
1164,1705204.461806615,1315350.7866327388,2.0104360604699993E9,1.550798577439999E9,1.2963876094010562,0.29638760940105624


Nessa análise geral houveram 1164 candidatos recorrentes em 2022 que concorreram em 2018 que tiveram algum valor declarado. Estes com variação total positiva de aproximadamente 30%, seguindo a média declarada com aumento, seguindo a analise feita com todos os candidatos.

In [0]:
%sql
--Primeira comparação entre 2020 e 2022
SELECT
   COUNT(DISTINCT NR_CPF_CANDIDATO)
  ,AVG(VR_BEM_CANDIDATO) AS MEDIA_BENS_2022
  ,AVG(VR_BEM_CANDIDATO_2020) AS MEDIA_BENS_2020
  ,SUM(VR_BEM_CANDIDATO) AS VR_BEM_CANDIDATO_2022
  ,SUM(VR_BEM_CANDIDATO_2020) AS VR_BEM_CANDIDATO_2020
  ,SUM(VR_BEM_CANDIDATO) / SUM(VR_BEM_CANDIDATO_2020) AS VARIACAO_22_20
  ,SUM(VR_BEM_CANDIDATO) / SUM(VR_BEM_CANDIDATO_2020) - 1  AS PERC_VARIACAO_22_20
FROM 
  eleicao_gold.patrimonio
WHERE 
  DS_CARGO IS NOT NULL AND DS_CARGO_2020 IS NOT NULL AND VR_BEM_CANDIDATO IS NOT NULL AND VR_BEM_CANDIDATO_2020 IS NOT NULL

count(DISTINCT NR_CPF_CANDIDATO),MEDIA_BENS_2022,MEDIA_BENS_2020,VR_BEM_CANDIDATO_2022,VR_BEM_CANDIDATO_2020,VARIACAO_22_20,PERC_VARIACAO_22_20
1306,659124.301655431,2845633.373722847,8.799309427100005E8,3.798920553920001E9,0.2316265713432791,-0.7683734286567209


Nessa análise geral houveram 1306 candidatos recorrentes em 2022 que concorreram em 2020(quase dobro mais que os que também concorreram em 2018). Estes com variação total negativa de aproximadamente 80%, seguido pela queda da média declarada.

Vamos começar algumas quebras, usando os anos de 2018 e 2022

In [0]:
%sql
--Comparação entre 2022 e 2018 por cargos concorridos em 2022
SELECT
   DS_CARGO
  ,COUNT(DISTINCT NR_CPF_CANDIDATO)
  ,AVG(COALESCE(VR_BEM_CANDIDATO,0)) AS MEDIA_BENS_2022
  ,AVG(COALESCE(VR_BEM_CANDIDATO_2018,0)) AS MEDIA_BENS_2018
  ,SUM(COALESCE(VR_BEM_CANDIDATO,0)) AS VR_BEM_CANDIDATO_2022
  ,SUM(COALESCE(VR_BEM_CANDIDATO_2018,0)) AS VR_BEM_CANDIDATO_2018
  ,SUM(COALESCE(VR_BEM_CANDIDATO,0)) / SUM(COALESCE(VR_BEM_CANDIDATO_2018,0)) AS VARIACAO_22_18
  ,(SUM(COALESCE(VR_BEM_CANDIDATO,0)) / SUM(COALESCE(VR_BEM_CANDIDATO_2018,0)) - 1) * 100 AS PERC_VARIACAO_22_18 -- mostrando em pontos percentuais
FROM 
  eleicao_gold.patrimonio
WHERE 
  DS_CARGO IS NOT NULL AND DS_CARGO_2018 IS NOT NULL AND VR_BEM_CANDIDATO IS NOT NULL AND VR_BEM_CANDIDATO_2018 IS NOT NULL
GROUP BY 
  1

DS_CARGO,count(DISTINCT NR_CPF_CANDIDATO),MEDIA_BENS_2022,MEDIA_BENS_2018,VR_BEM_CANDIDATO_2022,VR_BEM_CANDIDATO_2018,VARIACAO_22_18,PERC_VARIACAO_22_18
VICE-GOVERNADOR,16,6197461.87625,4996333.610625,9.915939002E7,7.994133777E7,1.2404019345447088,24.04019345447088
SENADOR,28,2818303.7078571427,2431601.1635714285,7.891250382E7,6.808483258E7,1.1590320608819509,15.903206088195088
1º SUPLENTE,7,1480463.6414285714,1192347.007142857,1.036324549E7,8346429.049999999,1.2416382416861258,24.163824168612578
DEPUTADO FEDERAL,420,2181570.4350000005,1388541.5006839628,9.249858644400003E8,5.887415962900002E8,1.571123681881608,57.11236818816079
GOVERNADOR,32,8626464.3059375,7359143.2365625,2.7604685779E8,2.3549258357E8,1.1722104093691992,17.22104093691992
2º SUPLENTE,10,807433.7827272727,728020.1681818181,8881771.61,8008221.85,1.1090816134170909,10.90816134170909
DEPUTADO ESTADUAL,532,872504.1648148148,839906.2292592594,4.71152249E8,4.535493638000001E8,1.0388113987251941,3.8811398725194124
DEPUTADO DISTRITAL,121,1164745.2752066113,897803.409338843,1.4093417829999995E8,1.0863421253E8,1.2973277480248695,29.73277480248695


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Como percebido, aumento distribuido por cargos disputados de 4% a 57%

In [0]:
%sql
--Comparação entre 2022 e 2020 por cargos concorridos em 2022
SELECT
   DS_CARGO
  ,COUNT(DISTINCT NR_CPF_CANDIDATO) AS QTD_CANDIDATOS
  ,AVG(COALESCE(VR_BEM_CANDIDATO,0)) AS MEDIA_BENS_2022
  ,AVG(COALESCE(VR_BEM_CANDIDATO_2018,0)) AS MEDIA_BENS_2020
  ,SUM(COALESCE(VR_BEM_CANDIDATO,0)) AS VR_BEM_CANDIDATO_2022
  ,SUM(COALESCE(VR_BEM_CANDIDATO_2020,0)) AS VR_BEM_CANDIDATO_2020
  ,SUM(COALESCE(VR_BEM_CANDIDATO,0)) / SUM(COALESCE(VR_BEM_CANDIDATO_2020,0)) AS VARIACAO_22_20
  ,(SUM(COALESCE(VR_BEM_CANDIDATO,0)) / SUM(COALESCE(VR_BEM_CANDIDATO_2020,0)) - 1) * 100 AS PERC_VARIACAO_22_20 -- mostrando em pontos percentuais
FROM 
  eleicao_gold.patrimonio
WHERE 
  DS_CARGO IS NOT NULL AND DS_CARGO_2020 IS NOT NULL AND VR_BEM_CANDIDATO IS NOT NULL AND VR_BEM_CANDIDATO_2020 IS NOT NULL
GROUP BY 
  1

DS_CARGO,QTD_CANDIDATOS,MEDIA_BENS_2022,MEDIA_BENS_2020,VR_BEM_CANDIDATO_2022,VR_BEM_CANDIDATO_2020,VARIACAO_22_20,PERC_VARIACAO_22_20
VICE-GOVERNADOR,4,334560.62,232475.0,1338242.48,1160570.17,1.1530905365248187,15.309053652481875
SENADOR,9,5948533.615555555,4271307.445555556,5.353680253999999E7,4.428993846E7,1.208780242229309,20.878024222930904
1º SUPLENTE,6,974158.2166666667,1923.3033333333333,5844949.3,3853125.1999999997,1.5169372902806273,51.69372902806273
DEPUTADO FEDERAL,493,693388.5050395259,185845.40444664026,3.5085458355000013E8,3.030169233299999E8,1.157871249217004,15.787124921700402
GOVERNADOR,14,635884.72,351635.6492857143,8902386.08,9510695.329999998,0.9360394556977152,-6.396054430228482
2º SUPLENTE,7,2960868.1900000004,69864.28571428571,2.0726077330000002E7,1.410584323E7,1.469325654060881,46.9325654060881
DEPUTADO ESTADUAL,773,556179.6999872773,245027.71882951653,4.3715724418999994E8,3.4225784582000017E9,0.1277274573918487,-87.22725426081513
DEPUTADO DISTRITAL,3,523552.41333333333,0.0,1570657.24,405000.0,3.878166024691358,287.81660246913583


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Aqui percebemos uma variação maior na abertura por cargos, onde a maior concentração de candidatos recorrentes (Deputado Estadual) teve a maior queda (87%) e a menor concentração (Deputado Distrital) teve o maior aumento (~290%)

OBS: Apenas para conhecimento, Deputado Distrital é um cargo exclusivo da Capital do país Brasília, suas atribuições são semelhantes as dos Deputados Estaduais.

Agora a abertura por situação final na eleição anterior, avaliando candidatos eleitos _versus_ não eleitos e demais situações finais.

In [0]:
%sql
--Comparação entre 2022 e 2018 por situação final da eleição
SELECT
   DS_SIT_TOT_TURNO_2018
  ,COUNT(DISTINCT NR_CPF_CANDIDATO) AS QTD_CANDIDATOS
  ,AVG(COALESCE(VR_BEM_CANDIDATO,0)) AS MEDIA_BENS_2022
  ,AVG(COALESCE(VR_BEM_CANDIDATO_2018,0)) AS MEDIA_BENS_2018
  ,SUM(COALESCE(VR_BEM_CANDIDATO,0)) AS VR_BEM_CANDIDATO_2022
  ,SUM(COALESCE(VR_BEM_CANDIDATO_2018,0)) AS VR_BEM_CANDIDATO_2018
  ,SUM(COALESCE(VR_BEM_CANDIDATO,0)) / SUM(COALESCE(VR_BEM_CANDIDATO_2018,0)) AS VARIACAO_22_18
  ,(SUM(COALESCE(VR_BEM_CANDIDATO,0)) / SUM(COALESCE(VR_BEM_CANDIDATO_2018,0)) - 1) * 100 AS PERC_VARIACAO_22_18 -- mostrando em pontos percentuais
FROM 
  eleicao_gold.patrimonio
WHERE 
  DS_CARGO IS NOT NULL AND DS_CARGO_2018 IS NOT NULL AND VR_BEM_CANDIDATO IS NOT NULL AND VR_BEM_CANDIDATO_2018 IS NOT NULL
GROUP BY 
  1

DS_SIT_TOT_TURNO_2018,count(DISTINCT NR_CPF_CANDIDATO),MEDIA_BENS_2022,MEDIA_BENS_2018,VR_BEM_CANDIDATO_2022,VR_BEM_CANDIDATO_2018,VARIACAO_22_18,PERC_VARIACAO_22_18
ELEITO POR MÉDIA,79,1844636.7182278482,999552.8543037974,1.4572630074E8,7.896467549E7,1.8454619085777746,84.54619085777746
ELEITO,24,1.21428298025E7,1.0784285346666666E7,2.9142791526E8,2.5882284832E8,1.1259744537688117,12.597445376881167
NÃO ELEITO,216,2644269.4690867597,1564963.8600456621,5.790950137300004E8,3.4272708535E8,1.6896680725966449,68.96680725966449
SUPLENTE,550,869418.7829107145,836513.3651249998,4.868745184300001E8,4.6844748446999985E8,1.039336392169655,3.933639216965501
#NULO#,20,967462.9029999999,1536898.9819999998,1.934925806E7,3.0737979639999997E7,0.6294902360733037,-37.05097639266963
ELEITO POR QP,276,1761599.4738267146,1339705.7912274373,4.8796305424999994E8,3.7109850417000014E8,1.314915174183683,31.49151741836831


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Eleitos por média com maior variação (85%) e status "NULO" significa que a informação não estava disponível no banco de dados(segundo documentação), explicada abaixo.  

In [0]:
%sql
SELECT 
  DISTINCT 
   DS_SIT_TOT_TURNO_2018
  ,DS_SITUACAO_CANDIDATURA_2018
FROM 
  eleicao_gold.patrimonio
WHERE 
  DS_CARGO IS NOT NULL AND DS_CARGO_2018 IS NOT NULL AND VR_BEM_CANDIDATO IS NOT NULL AND VR_BEM_CANDIDATO_2018 IS NOT NULL


DS_SIT_TOT_TURNO_2018,DS_SITUACAO_CANDIDATURA_2018
ELEITO POR QP,APTO
NÃO ELEITO,INAPTO
#NULO#,INAPTO
SUPLENTE,APTO
NÃO ELEITO,APTO
ELEITO POR MÉDIA,APTO
ELEITO,APTO
SUPLENTE,INAPTO


Como mostrado os status "#NULO#" são candidatos inaptos a concorrerem no ano em questão.

In [0]:
%sql
--Comparação entre 2022 e 2020 por situação final da eleição 
SELECT
   DS_SIT_TOT_TURNO_2020
  ,COUNT(DISTINCT NR_CPF_CANDIDATO) AS QTD_CANDIDATOS
  ,AVG(COALESCE(VR_BEM_CANDIDATO,0)) AS MEDIA_BENS_2022
  ,AVG(COALESCE(VR_BEM_CANDIDATO_2020,0)) AS MEDIA_BENS_2020
  ,SUM(COALESCE(VR_BEM_CANDIDATO,0)) AS VR_BEM_CANDIDATO_2022
  ,SUM(COALESCE(VR_BEM_CANDIDATO_2020,0)) AS VR_BEM_CANDIDATO_2020
  ,SUM(COALESCE(VR_BEM_CANDIDATO,0)) / SUM(COALESCE(VR_BEM_CANDIDATO_2020,0)) AS VARIACAO_22_20
  ,(SUM(COALESCE(VR_BEM_CANDIDATO,0)) / SUM(COALESCE(VR_BEM_CANDIDATO_2020,0)) - 1) * 100 AS PERC_VARIACAO_22_20 -- mostrando em pontos percentuais
FROM 
  eleicao_gold.patrimonio
WHERE 
  DS_CARGO IS NOT NULL AND DS_CARGO_2020 IS NOT NULL AND VR_BEM_CANDIDATO IS NOT NULL AND VR_BEM_CANDIDATO_2020 IS NOT NULL
GROUP BY 
  1

DS_SIT_TOT_TURNO_2020,QTD_CANDIDATOS,MEDIA_BENS_2022,MEDIA_BENS_2020,VR_BEM_CANDIDATO_2022,VR_BEM_CANDIDATO_2020,VARIACAO_22_20,PERC_VARIACAO_22_20
ELEITO POR MÉDIA,80,656714.6503614456,546310.511686747,5.450731597999999E7,4.534377247E7,1.2020904527972989,20.209045279729885
ELEITO,35,722837.0114285715,634693.3268571428,2.5299295400000002E7,2.221426644E7,1.1388760222324947,13.887602223249473
NÃO ELEITO,548,833994.9938530463,1100543.5379390682,4.653692065699999E8,6.141032941700001E8,0.757802817519447,-24.2197182480553
SUPLENTE,458,466354.05590618344,6442170.782324093,2.1872005222000003E8,3.02137809691E9,0.07239082471792845,-92.76091752820716
#NULO#,20,466335.63500000007,584789.8085,9326712.700000001,1.1695796170000002E7,0.7974414537013943,-20.255854629860572
ELEITO POR QP,169,627696.2343529413,495207.81035294104,1.0670835984000002E8,8.418532775999998E7,1.2675410630247814,26.754106302478142


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Eleitos por QP (Cosciente Partidário) tiveram a maior variação (27%) enquanto a menor foi dos eleitos a suplente (-93%) de variação patrimonial.

OBS: Para conhecimento, "ELEITO POR QP" (Cosciente Partidário) são as vagas destinadas a candidatos eleitos por eleitores que optaram por votar na sigla do partido, mais informações no [LINK](https://www.tse.jus.br/comunicacao/noticias/2024/Marco/quocientes-eleitoral-e-partidario-entenda-como-um-candidato-a-vereador-e-eleito)

In [0]:
%sql
--Comparação entre 2022 e 2018 por genero declarado em 2022
SELECT
   DS_GENERO
  ,COUNT(DISTINCT NR_CPF_CANDIDATO) AS QTD_CANDIDATOS
  ,AVG(COALESCE(VR_BEM_CANDIDATO,0)) AS MEDIA_BENS_2022
  ,AVG(COALESCE(VR_BEM_CANDIDATO_2018,0)) AS MEDIA_BENS_2018
  ,SUM(COALESCE(VR_BEM_CANDIDATO,0)) AS VR_BEM_CANDIDATO_2022
  ,SUM(COALESCE(VR_BEM_CANDIDATO_2018,0)) AS VR_BEM_CANDIDATO_2018
  ,SUM(COALESCE(VR_BEM_CANDIDATO,0)) / SUM(COALESCE(VR_BEM_CANDIDATO_2018,0)) AS VARIACAO_22_18
  ,(SUM(COALESCE(VR_BEM_CANDIDATO,0)) / SUM(COALESCE(VR_BEM_CANDIDATO_2018,0)) - 1) * 100 AS PERC_VARIACAO_22_18 -- mostrando em pontos percentuais
FROM 
  eleicao_gold.patrimonio
WHERE 
  DS_CARGO IS NOT NULL AND DS_CARGO_2018 IS NOT NULL AND VR_BEM_CANDIDATO IS NOT NULL AND VR_BEM_CANDIDATO_2018 IS NOT NULL
GROUP BY 
  1

DS_GENERO,QTD_CANDIDATOS,MEDIA_BENS_2022,MEDIA_BENS_2018,VR_BEM_CANDIDATO_2022,VR_BEM_CANDIDATO_2018,VARIACAO_22_18,PERC_VARIACAO_22_18
MASCULINO,946,1880733.6863350773,1431820.280157067,1.7961006704499989E9,1.367388367549999E9,1.3135263638874886,31.352636388748856
FEMININO,218,956854.4197321429,818795.5798660715,2.1433539002E8,1.8341020989000002E8,1.1686120971594074,16.86120971594074


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Maior variação do Genero Masculino (31%) _versus_ (17%) Genero

In [0]:
%sql
--Comparação entre 2022 e 2020 por genero declarado em 2022
SELECT
   DS_GENERO
  ,COUNT(DISTINCT NR_CPF_CANDIDATO) AS QTD_CANDIDATOS
  ,AVG(COALESCE(VR_BEM_CANDIDATO,0)) AS MEDIA_BENS_2022
  ,AVG(COALESCE(VR_BEM_CANDIDATO_2020,0)) AS MEDIA_BENS_2020
  ,SUM(COALESCE(VR_BEM_CANDIDATO,0)) AS VR_BEM_CANDIDATO_2022
  ,SUM(COALESCE(VR_BEM_CANDIDATO_2020,0)) AS VR_BEM_CANDIDATO_2020
  ,SUM(COALESCE(VR_BEM_CANDIDATO,0)) / SUM(COALESCE(VR_BEM_CANDIDATO_2020,0)) AS VARIACAO_22_20
  ,(SUM(COALESCE(VR_BEM_CANDIDATO,0)) / SUM(COALESCE(VR_BEM_CANDIDATO_2020,0)) - 1) * 100 AS PERC_VARIACAO_22_20 -- mostrando em pontos percentuais
FROM 
  eleicao_gold.patrimonio
WHERE 
  DS_CARGO IS NOT NULL AND DS_CARGO_2020 IS NOT NULL AND VR_BEM_CANDIDATO IS NOT NULL AND VR_BEM_CANDIDATO_2020 IS NOT NULL
GROUP BY 
  1

DS_GENERO,QTD_CANDIDATOS,MEDIA_BENS_2022,MEDIA_BENS_2020,VR_BEM_CANDIDATO_2022,VR_BEM_CANDIDATO_2020,VARIACAO_22_20,PERC_VARIACAO_22_20
MASCULINO,948,757022.5324355008,3600217.0662229103,7.335548339300002E8,3.48861033717E9,0.2102713582294399,-78.97286417705601
FEMININO,358,399934.7234426227,847842.1222677598,1.463761087799999E8,3.1031021675000006E8,0.47170895729136475,-52.829104270863525


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Queda de Patrimonio nos dois Generos entre as eleições de 2020 e 2022

Por fim, mas não menos importante, distinção por idade:<br>
Nesta avaliação farei intervalos de idade (0-18, 19-30, 31-40, 41-50, 51-60, 60-70, >70)

In [0]:
%sql
--Comparação entre 2022 e 2018 por idade em 2022
SELECT
   CASE 
    WHEN DATEDIFF ( YEAR, DT_NASCIMENTO, DT_ELEICAO) <= 18 THEN '1. 18'
    WHEN DATEDIFF ( YEAR, DT_NASCIMENTO, DT_ELEICAO) <= 30 THEN '2. 19-30'
    WHEN DATEDIFF ( YEAR, DT_NASCIMENTO, DT_ELEICAO) <= 40 THEN '3. 31-40'
    WHEN DATEDIFF ( YEAR, DT_NASCIMENTO, DT_ELEICAO) <= 50 THEN '4. 41-50'
    WHEN DATEDIFF ( YEAR, DT_NASCIMENTO, DT_ELEICAO) <= 50 THEN '5. 51-60'
    WHEN DATEDIFF ( YEAR, DT_NASCIMENTO, DT_ELEICAO) <= 70 THEN '6. 61-70'
    WHEN DATEDIFF ( YEAR, DT_NASCIMENTO, DT_ELEICAO) > 70 THEN '7. 70+'
    END AS IDADE_CANDIDATO
  ,COUNT(DISTINCT NR_CPF_CANDIDATO) AS QTD_CANDIDATOS
  ,AVG(COALESCE(VR_BEM_CANDIDATO,0)) AS MEDIA_BENS_2022
  ,AVG(COALESCE(VR_BEM_CANDIDATO_2018,0)) AS MEDIA_BENS_2018
  ,SUM(COALESCE(VR_BEM_CANDIDATO,0)) AS VR_BEM_CANDIDATO_2022
  ,SUM(COALESCE(VR_BEM_CANDIDATO_2018,0)) AS VR_BEM_CANDIDATO_2018
  ,SUM(COALESCE(VR_BEM_CANDIDATO,0)) / SUM(COALESCE(VR_BEM_CANDIDATO_2018,0)) AS VARIACAO_22_18
  ,(SUM(COALESCE(VR_BEM_CANDIDATO,0)) / SUM(COALESCE(VR_BEM_CANDIDATO_2018,0)) - 1) * 100 AS PERC_VARIACAO_22_18 -- mostrando em pontos percentuais
FROM 
  eleicao_gold.patrimonio
WHERE 
  DS_CARGO IS NOT NULL AND DS_CARGO_2018 IS NOT NULL AND VR_BEM_CANDIDATO IS NOT NULL AND VR_BEM_CANDIDATO_2020 IS NOT NULL
GROUP BY 
  1

IDADE_CANDIDATO,QTD_CANDIDATOS,MEDIA_BENS_2022,MEDIA_BENS_2018,VR_BEM_CANDIDATO_2022,VR_BEM_CANDIDATO_2018,VARIACAO_22_18,PERC_VARIACAO_22_18
3. 31-40,50,377168.2186,242977.44459999996,1.885841093E7,1.2148872229999999E7,1.5522766700461053,55.227667004610524
7. 70+,11,1087631.6209090909,920901.8745454546,1.196394783E7,1.0129920620000001E7,1.1810505016573367,18.105050165733672
6. 61-70,218,1017593.6402232145,1099709.5249107142,2.2794097541000006E8,2.4633493357999998E8,0.9253294776234964,-7.467052237650362
4. 41-50,139,469951.44267605635,440010.65838028176,6.673310486E7,6.248151349000001E7,1.0680455887273033,6.804558872730326
2. 19-30,6,455104.88666666666,54912.94166666667,2730629.32,329477.65,8.287752811154261,728.7752811154261


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Grande aumento entre 2018 e 2022 nos candidatos entre 19-30 anos (278%) e queda entre candidatos de 61-70 anos (-7%).

In [0]:
%sql
--Comparação entre 2022 e 2020 por idade em 2022
SELECT
   CASE 
    WHEN DATEDIFF ( YEAR, DT_NASCIMENTO, DT_ELEICAO) <= 18 THEN '1. 18'
    WHEN DATEDIFF ( YEAR, DT_NASCIMENTO, DT_ELEICAO) <= 30 THEN '2. 19-30'
    WHEN DATEDIFF ( YEAR, DT_NASCIMENTO, DT_ELEICAO) <= 40 THEN '3. 31-40'
    WHEN DATEDIFF ( YEAR, DT_NASCIMENTO, DT_ELEICAO) <= 50 THEN '4. 41-50'
    WHEN DATEDIFF ( YEAR, DT_NASCIMENTO, DT_ELEICAO) <= 50 THEN '5. 51-60'
    WHEN DATEDIFF ( YEAR, DT_NASCIMENTO, DT_ELEICAO) <= 70 THEN '6. 61-70'
    WHEN DATEDIFF ( YEAR, DT_NASCIMENTO, DT_ELEICAO) > 70 THEN '7. 70+'
    END AS IDADE_CANDIDATO
  ,COUNT(DISTINCT NR_CPF_CANDIDATO) AS QTD_CANDIDATOS
  ,AVG(COALESCE(VR_BEM_CANDIDATO,0)) AS MEDIA_BENS_2022
  ,AVG(COALESCE(VR_BEM_CANDIDATO_2020,0)) AS MEDIA_BENS_2020
  ,SUM(COALESCE(VR_BEM_CANDIDATO,0)) AS VR_BEM_CANDIDATO_2022
  ,SUM(COALESCE(VR_BEM_CANDIDATO_2020,0)) AS VR_BEM_CANDIDATO_2020
  ,SUM(COALESCE(VR_BEM_CANDIDATO,0)) / SUM(COALESCE(VR_BEM_CANDIDATO_2020,0)) AS VARIACAO_22_20
  ,(SUM(COALESCE(VR_BEM_CANDIDATO,0)) / SUM(COALESCE(VR_BEM_CANDIDATO_2020,0)) - 1) * 100 AS PERC_VARIACAO_22_20 -- mostrando em pontos percentuais
FROM 
  eleicao_gold.patrimonio
WHERE 
  DS_CARGO IS NOT NULL AND DS_CARGO_2020 IS NOT NULL AND VR_BEM_CANDIDATO IS NOT NULL AND VR_BEM_CANDIDATO_2020 IS NOT NULL
GROUP BY 
  1

IDADE_CANDIDATO,QTD_CANDIDATOS,MEDIA_BENS_2022,MEDIA_BENS_2020,VR_BEM_CANDIDATO_2022,VR_BEM_CANDIDATO_2020,VARIACAO_22_20,PERC_VARIACAO_22_20
3. 31-40,224,407705.83913043485,334592.3681739129,9.377234300000001E7,7.695624467999998E7,1.2185150586534577,21.851505865345764
7. 70+,26,1038637.4230769231,1088240.2365384614,27004573,2.829424615E7,0.9544192432919794,-4.558075670802064
6. 61-70,570,861605.8624872231,5647186.402282796,5.0576264128E8,3.3148984181400013E9,0.1525725912179791,-84.7427408782021
4. 41-50,448,538001.1398458149,821328.1728634362,2.4425251748999995E8,3.7288299048E8,0.6550379709612973,-34.49620290387026
2. 19-30,38,240496.52473684208,154964.5913157895,9138867.94,5888654.470000001,1.5519450133402035,55.19450133402035


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Variação entre 2020 e 2022 nos candidatos entre 19-30 anos (55%) e queda entre candidatos de 61-70 anos (-85%)


### Conclusão:

Concluindo a análise com as respostas apresentadas acima, a ideia inicial do projeto era avalias mais anos, provavelmente com diferentes metodologias e abordagens, por tempo optei em analisar apenas esse 3 anos tendo base o ano de 2022.<br>
Em 2024 uma normativa tornou não público os CPF's dos candidatos, o que faria a união das tabelas entre anos muito mais complicadas, tendo _join_ apenas pelo nome dos candidatos.<br>
A base é bem vasta permitindo outros diversos desdobramentos não explorados aqui, como cargo anterior dos candidatos, abrir maiores intervalos de idade dadas as concentrações por intervalo definido, ou até mesmo combinar as aberturas como por exemplo idade por situação de eleição, etc.<br>
Como o foco do projeto era o tratamento e a manipulação dos dados, me dediquei mais a esses pontos. 